In [1]:
# Google Colab 환경에서 작성
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Library

In [2]:
import os
import pandas as pd
import numpy as np

import random as python_random 

import matplotlib.pyplot as plt
%matplotlib inline

import missingno as msno
import seaborn as sns

# scikit-learn과 catboost install
!pip install --upgrade scikit-learn
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 18.9 MB/s eta 0:00:00


# Version

In [3]:
import platform

# OS
platform.platform()

'Linux-5.10.147+-x86_64-with-glibc2.29'

In [4]:
# 런타임 유형 : CPU
print('런타임 유형 : CPU')
print()
!cat /proc/cpuinfo

런타임 유형 : CPU

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0xffffffff
cpu MHz		: 2200.146
cache size	: 56320 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa mmio_stale_data retbleed
bogomips	: 4400.29
clflush size	: 64
cache_alignme

In [5]:
import sys
import sklearn
import numpy
import pandas
import catboost

sys.version
print('-Google Colab-')
!cat /etc/issue.net
!python --version
print('Numpy ' + numpy.__version__)
print('Pandas ' + pandas.__version__)
print('Sklearn ' + sklearn.__version__)
print('CatBoost ' + catboost.__version__)

-Google Colab-
Ubuntu 20.04.5 LTS
Python 3.8.10
Numpy 1.22.4
Pandas 1.3.5
Sklearn 1.2.1
CatBoost 1.1.1


# Parameter

In [6]:
PAR = {
    'SEED' : 42,
    'FOLD' : 6,
    'EARLY_STOP' : 1000,
    'FILEPATH' : '/content/drive/MyDrive/LG Comp/LG_data.zip', # zip파일 데이터셋 경로
    'TRAIN_DIRECTPATH' : '', # train.csv 데이터셋 직접 경로  './data/train.csv'
    'TEST_DIRECTPATH' : '', # test.csv 데이터셋 직접 경로  './data/test.csv'
    'SUB_DIRECTPATH' : '', # sample_submission.csv 데이터셋 직접 경로  './data/sample_submission.csv'
    'OUTPUT_PATH' : './' # 출력물이 저장되는 경로  './data/'
}

#SEED

In [7]:
# 시드 고정
seed_num = PAR['SEED']
np.random.seed(seed_num)
python_random.seed(seed_num)

# Dataset

In [8]:
# ZIP 경로만 설정 시(변경 필수)
!cp '/content/drive/MyDrive/LG Comp/LG_data.zip' 'tmp.zip' # 데이터셋 zip 파일 경로 지정 후 현재 디렉터리에 임시 zip파일 생성(zip파일 경로 입력)
!unzip "tmp.zip" # tmp.zip을 현재 디렉터리에 압축해제

# CSV 직접 경로 설정 시
if PAR['TRAIN_DIRECTPATH'] and PAR['TEST_DIRECTPATH'] and PAR['SUB_DIRECTPATH'] != '' :
  train = pd.read_csv(PAR['TRAIN_DIRECTPATH'])
  test = pd.read_csv(PAR['TEST_DIRECTPATH'])
  sub = pd.read_csv(PAR['SUB_DIRECTPATH'])

# Colab 환경에서 ZIP 경로만 설정 시
elif PAR['FILEPATH'].split('/')[0]=='content' : 
  train = pd.read_csv('/content/train.csv')
  test = pd.read_csv('/content/test.csv')
  sub = pd.read_csv('/content/sample_submission.csv')

# 이외 ./ 경로
else :
  train = pd.read_csv('./train.csv')
  test = pd.read_csv('./test.csv')
  sub = pd.read_csv('./sample_submission.csv')

Archive:  tmp.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [9]:
# view
pd.options.display.max_columns=100
display(train.head(5),test.head(5))

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_23,X_24,X_25,X_26,X_27,X_28,X_29,X_30,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40,X_41,X_42,X_43,X_44,...,X_2826,X_2827,X_2828,X_2829,X_2830,X_2831,X_2832,X_2833,X_2834,X_2835,X_2836,X_2837,X_2838,X_2839,X_2840,X_2841,X_2842,X_2843,X_2844,X_2845,X_2846,X_2847,X_2848,X_2849,X_2850,X_2851,X_2852,X_2853,X_2854,X_2855,X_2856,X_2857,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,52.6,52.6,52.0,53.2,52.3,52.3,58.7,55.4,50.5,50.5,50.1,9.04,NaN,5.02,7.00,40.06,0.000331,0.000033,NaN,47.2,47.094444,47.0,39.0,32.722222,26.0,184.7,175.903333,164.9,201.2,191.800000,181.6,139.6,131.646667,115.4,209.0,197.286667,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,53.8,53.8,57.0,56.7,54.4,54.4,61.1,56.4,52.0,52.0,51.9,8.00,NaN,5.00,7.05,36.54,0.001150,0.000009,NaN,47.1,47.084906,47.0,34.0,30.849057,27.0,186.0,168.417241,151.2,194.9,188.755172,177.1,145.5,128.748276,119.7,198.0,193.296552,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,52.6,52.6,52.0,53.2,52.3,52.3,58.7,55.4,50.5,50.5,50.1,9.04,NaN,5.02,6.52,40.05,0.000332,0.000033,NaN,47.2,47.088679,47.0,35.0,30.603774,27.0,181.4,176.137931,167.4,194.5,187.310345,183.6,128.0,115.365517,104.0,193.4,179.820690,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,53.8,53.8,57.0,56.7,54.4,54.4,61.1,56.4,52.0,52.0,51.9,7.49,NaN,5.01,7.03,40.03,0.001210,0.000008,NaN,47.2,47.105660,46.9,38.0,32.698113,27.0,180.5,173.637931,166.7,193.0,187.203448,179.5,126.2,112.082759,94.5,190.3,181.920690,165.8,384.0,369.188679,353.0,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,52.6,52.6,52.0,53.2,52.3,52.3,58.7,55.4,50.5,50.5,50.1,9.04,NaN,5.03,6.52,40.07,0.000334,0.000041,NaN,47.2,47.107407,47.0,36.0,32.888889,28.0,184.6,175.366667,167.6,198.1,194.546667,188.0,152.1,138.066667,109.7,208.6,196.393333,182.6,383.0,367.351852,352.0,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


,PRODUCT_ID,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_23,X_24,X_25,X_26,X_27,X_28,X_29,X_30,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40,X_41,X_42,X_43,X_44,X_45,X_46,...,X_2826,X_2827,X_2828,X_2829,X_2830,X_2831,X_2832,X_2833,X_2834,X_2835,X_2836,X_2837,X_2838,X_2839,X_2840,X_2841,X_2842,X_2843,X_2844,X_2845,X_2846,X_2847,X_2848,X_2849,X_2850,X_2851,X_2852,X_2853,X_2854,X_2855,X_2856,X_2857,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TEST_000,2022-09-09 2:01,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,51.0,10.0,52.0,2.0,469.6,474.4,461.0,0.0,0.0,247.5,248.0,247.2,250.0,526.0,537.0,521.0,0.19,0.19,0.19,155.0,155.0,155.0,155.0,185.0,185.0,185.0,185.0,155.0,155.0,155.0,155.0,0.075,2.0,67.2,68.9,67.5,79.8,70.9,80.5,71.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TEST_001,2022-09-09 2:09,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,2.0,506.6,511.1,500.1,0.0,0.0,247.6,248.2,247.4,250.0,489.0,496.0,484.0,0.19,0.19,0.19,155.0,155.0,155.0,155.0,185.0,185.0,185.0,185.0,155.0,155.0,155.0,155.0,0.079,2.0,63.7,64.6,64.7,78.6,69.0,82.2,70.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TEST_002,2022-09-09 8:42,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,2.0,506.6,511.4,500.6,0.0,0.0,247.6,247.9,247.4,250.0,488.0,496.0,484.0,0.19,0.19,0.19,155.0,155.0,155.0,155.0,185.0,185.0,185.0,185.0,155.0,155.0,155.0,155.0,0.079,2.0,63.7,64.6,64.7,78.6,69.0,82.2,70.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TEST_003,2022-09-09 10:56,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1,2.686538,2.3,40.0,34.519231,28.0,168.9,160.071429,150.8,184.6,169.696429,159.4,201.3,179.739286,149.9,198.0,191.450000,183.8,467.0,444.192308,423.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TEST_004,2022-09-09 11:04,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1,2.732692,2.5,37.0,33.807692,29.0,170.6,164.532143,159.7,187.8,183.171429,175.3,227.9,190.664286,162.4,210.2,193.082143,179.7,465.0,443.211539,423.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Preprocessing

In [10]:
train2 = train.copy()
test2 = test.copy()

# TIMESTAMP 전처리
train2['TIMESTAMP'] = pd.to_datetime(train2['TIMESTAMP'])
train2['month'] = train2['TIMESTAMP'].dt.month # 월
train2['day'] = train2['TIMESTAMP'].dt.day # 일
train2['hour'] = train2['TIMESTAMP'].dt.hour # 시간
train2['minute'] = train2['TIMESTAMP'].dt.minute # 분

test2['TIMESTAMP'] = pd.to_datetime(test2['TIMESTAMP'])
test2['month'] = test2['TIMESTAMP'].dt.month # 월
test2['day'] = test2['TIMESTAMP'].dt.day # 일
test2['hour'] = test2['TIMESTAMP'].dt.hour # 시간
test2['minute'] = test2['TIMESTAMP'].dt.minute # 분

# Label
label_df = train['Y_Class']

# 학습에 불필요한 column drop
train2 = train2.drop(columns=['PRODUCT_ID','TIMESTAMP','Y_Class','Y_Quality'])
test2 = test2.drop(columns=['PRODUCT_ID','TIMESTAMP'])

# 전처리 후
display(train2.head(5),test2.head(5))

,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_23,X_24,X_25,X_26,X_27,X_28,X_29,X_30,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40,X_41,X_42,X_43,X_44,X_45,X_46,X_47,X_48,...,X_2830,X_2831,X_2832,X_2833,X_2834,X_2835,X_2836,X_2837,X_2838,X_2839,X_2840,X_2841,X_2842,X_2843,X_2844,X_2845,X_2846,X_2847,X_2848,X_2849,X_2850,X_2851,X_2852,X_2853,X_2854,X_2855,X_2856,X_2857,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875,month,day,hour,minute
0,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,52.3,52.3,58.7,55.4,50.5,50.5,50.1,9.04,NaN,5.02,7.00,40.06,0.000331,0.000033,NaN,47.2,47.094444,47.0,39.0,32.722222,26.0,184.7,175.903333,164.9,201.2,191.800000,181.6,139.6,131.646667,115.4,209.0,197.286667,189.0,383.0,368.296296,353.0,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN,6,13,5,14
1,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,54.4,54.4,61.1,56.4,52.0,52.0,51.9,8.00,NaN,5.00,7.05,36.54,0.001150,0.000009,NaN,47.1,47.084906,47.0,34.0,30.849057,27.0,186.0,168.417241,151.2,194.9,188.755172,177.1,145.5,128.748276,119.7,198.0,193.296552,185.6,383.0,367.735849,353.0,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN,6,13,5,22
2,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,52.3,52.3,58.7,55.4,50.5,50.5,50.1,9.04,NaN,5.02,6.52,40.05,0.000332,0.000033,NaN,47.2,47.088679,47.0,35.0,30.603774,27.0,181.4,176.137931,167.4,194.5,187.310345,183.6,128.0,115.365517,104.0,193.4,179.820690,165.5,383.0,367.320755,353.0,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN,6,13,5,30
3,T050307,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,54.4,54.4,61.1,56.4,52.0,52.0,51.9,7.49,NaN,5.01,7.03,40.03,0.001210,0.000008,NaN,47.2,47.105660,46.9,38.0,32.698113,27.0,180.5,173.637931,166.7,193.0,187.203448,179.5,126.2,112.082759,94.5,190.3,181.920690,165.8,384.0,369.188679,353.0,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN,6,13,5,39
4,T050304,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,52.3,52.3,58.7,55.4,50.5,50.5,50.1,9.04,NaN,5.03,6.52,40.07,0.000334,0.000041,NaN,47.2,47.107407,47.0,36.0,32.888889,28.0,184.6,175.366667,167.6,198.1,194.546667,188.0,152.1,138.066667,109.7,208.6,196.393333,182.6,383.0,367.351852,352.0,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN,6,13,5,47


,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,X_20,X_21,X_22,X_23,X_24,X_25,X_26,X_27,X_28,X_29,X_30,X_31,X_32,X_33,X_34,X_35,X_36,X_37,X_38,X_39,X_40,X_41,X_42,X_43,X_44,X_45,X_46,X_47,X_48,...,X_2830,X_2831,X_2832,X_2833,X_2834,X_2835,X_2836,X_2837,X_2838,X_2839,X_2840,X_2841,X_2842,X_2843,X_2844,X_2845,X_2846,X_2847,X_2848,X_2849,X_2850,X_2851,X_2852,X_2853,X_2854,X_2855,X_2856,X_2857,X_2858,X_2859,X_2860,X_2861,X_2862,X_2863,X_2864,X_2865,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875,month,day,hour,minute
0,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,51.0,10.0,52.0,2.0,469.6,474.4,461.0,0.0,0.0,247.5,248.0,247.2,250.0,526.0,537.0,521.0,0.19,0.19,0.19,155.0,155.0,155.0,155.0,185.0,185.0,185.0,185.0,155.0,155.0,155.0,155.0,0.075,2.0,67.2,68.9,67.5,79.8,70.9,80.5,71.2,67.0,76.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,9,2,1
1,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,2.0,506.6,511.1,500.1,0.0,0.0,247.6,248.2,247.4,250.0,489.0,496.0,484.0,0.19,0.19,0.19,155.0,155.0,155.0,155.0,185.0,185.0,185.0,185.0,155.0,155.0,155.0,155.0,0.079,2.0,63.7,64.6,64.7,78.6,69.0,82.2,70.4,64.7,74.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,9,2,9
2,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,45.0,10.0,31.0,2.0,506.6,511.4,500.6,0.0,0.0,247.6,247.9,247.4,250.0,488.0,496.0,484.0,0.19,0.19,0.19,155.0,155.0,155.0,155.0,185.0,185.0,185.0,185.0,155.0,155.0,155.0,155.0,0.079,2.0,63.7,64.6,64.7,78.6,69.0,82.2,70.4,64.7,74.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,9,8,42
3,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1,2.686538,2.3,40.0,34.519231,28.0,168.9,160.071429,150.8,184.6,169.696429,159.4,201.3,179.739286,149.9,198.0,191.450000,183.8,467.0,444.192308,423.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,9,10,56
4,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1,2.732692,2.5,37.0,33.807692,29.0,170.6,164.532143,159.7,187.8,183.171429,175.3,227.9,190.664286,162.4,210.2,193.082143,179.7,465.0,443.211539,423.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,9,11,4


# Model Fit & Predict

In [11]:
# Stratified_K-FOLD & CatBoost

from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier

skf = StratifiedKFold(n_splits = PAR['FOLD'], shuffle = True, random_state = PAR['SEED']) # Stratified 6-FOLD
cbc = CatBoostClassifier(objective='MultiClass', eval_metric='TotalF1:average=Macro', cat_features=['LINE','PRODUCT_CODE'], verbose=100) # CatBoostClassifier & cat_features, <random_state=PAR['SEED'] 적용해도 결과 동일>
result = 0
fold = 0

for train_index,valid_index in skf.split(train2,label_df):
    fold = fold + 1
    x_train=train2.iloc[train_index]
    x_valid=train2.iloc[valid_index]
    y_train=label_df.iloc[train_index]
    y_valid=label_df.iloc[valid_index]
    cbc.fit(x_train, y_train, eval_set=(x_valid,y_valid), early_stopping_rounds=PAR['EARLY_STOP'], verbose=100)
    result+=cbc.predict_proba(test2)/PAR['FOLD'] # Soft-Voting
    cbc.save_model(PAR['OUTPUT_PATH']+'{}_fold_model.cbm'.format(fold))

Learning rate set to 0.106481
0:	learn: 0.4489101	test: 0.4591631	best: 0.4591631 (0)	total: 357ms	remaining: 5m 56s
100:	learn: 0.8128513	test: 0.7119687	best: 0.7328529 (91)	total: 21.6s	remaining: 3m 12s
200:	learn: 0.9708098	test: 0.7824874	best: 0.7824874 (161)	total: 38s	remaining: 2m 31s
300:	learn: 1.0000000	test: 0.7824874	best: 0.7824874 (161)	total: 54.4s	remaining: 2m 6s
400:	learn: 1.0000000	test: 0.7824874	best: 0.7824874 (161)	total: 1m 10s	remaining: 1m 45s
500:	learn: 1.0000000	test: 0.7633403	best: 0.7824874 (161)	total: 1m 28s	remaining: 1m 27s
600:	learn: 1.0000000	test: 0.7867401	best: 0.8062290 (577)	total: 1m 45s	remaining: 1m 9s
700:	learn: 1.0000000	test: 0.8062290	best: 0.8062290 (577)	total: 2m 1s	remaining: 52s
800:	learn: 1.0000000	test: 0.8062290	best: 0.8062290 (577)	total: 2m 18s	remaining: 34.3s
900:	learn: 1.0000000	test: 0.8062290	best: 0.8062290 (577)	total: 2m 34s	remaining: 17s
999:	learn: 1.0000000	test: 0.8062290	best: 0.8062290 (577)	total: 2m 5

#Submission

In [12]:
sub['Y_Class'] = result.argmax(1) # Soft-Voting
display(sub)
print(sub['Y_Class'].value_counts())
sub.to_csv(PAR['OUTPUT_PATH']+'Submission.csv',index=0)

,PRODUCT_ID,Y_Class
0,TEST_000,1
1,TEST_001,1
2,TEST_002,1
3,TEST_003,1
4,TEST_004,1
...,...,...
305,TEST_305,1
306,TEST_306,1
307,TEST_307,1
308,TEST_308,1


1    268
0     37
2      5
Name: Y_Class, dtype: int64


# Model_Load

In [ ]:
# # Model Load & Predict (모델 로드 시)

# from catboost import CatBoostClassifier

# load_for_sub = sub.copy()
# load_result = 0

# for i in range(1,7) : 
#   load_model = CatBoostClassifier()
#   load_model.load_model(PAR['OUTPUT_PATH']+"{}_fold_model.cbm".format(i))
#   load_result+=load_model.predict_proba(test2)/PAR['FOLD'] # Soft-Voting

# load_for_sub['Y_Class'] = load_result.argmax(1) # Soft-Voting
# display(load_for_sub)
# load_for_sub.to_csv(PAR['OUTPUT_PATH']+'Load_Submission.csv',index=0)
# print(load_for_sub['Y_Class'].value_counts())